In [1]:
import pandas as pd
import numpy as np

In [7]:
n = np.random.random((3*100))
out = np.argmin(n.reshape((-1,3)), axis=1)

print(n.shape)
print(out.shape)

(300,)
(100,)


In [9]:
df = pd.read_clipboard()

In [12]:
df

,Date,Attributes,Symbols,value
11/12/2019,Adj,Close,AALR3.SA,18.001.112
11/12/2019,Adj,Close,ABCB4.SA,18.298.676
11/12/2019,Adj,Close,ABEV3.SA,17.977.827
12/12/2019,Adj,Close,AALR3.SA,18.211.111
12/12/2019,Adj,Close,ABCB4.SA,18.653.500
12/12/2019,Adj,Close,ABEV3.SA,18.005.134
13/12/2019,Adj,Close,AALR3.SA,18.100.537
13/12/2019,Adj,Close,ABCB4.SA,18.153.353
13/12/2019,Adj,Close,ABEV3.SA,18.369.147


In [14]:
grouped_dfs = [i[1] for i in df.groupby("Symbols")]

for i in grouped_dfs:
    print(i)
    print(' ')

           Date Attributes   Symbols       value
11/12/2019  Adj      Close  AALR3.SA  18.001.112
12/12/2019  Adj      Close  AALR3.SA  18.211.111
13/12/2019  Adj      Close  AALR3.SA  18.100.537
 
           Date Attributes   Symbols       value
11/12/2019  Adj      Close  ABCB4.SA  18.298.676
12/12/2019  Adj      Close  ABCB4.SA  18.653.500
13/12/2019  Adj      Close  ABCB4.SA  18.153.353
 
           Date Attributes   Symbols       value
11/12/2019  Adj      Close  ABEV3.SA  17.977.827
12/12/2019  Adj      Close  ABEV3.SA  18.005.134
13/12/2019  Adj      Close  ABEV3.SA  18.369.147
 


In [19]:
from timeit import repeat

In [15]:
import pandas as pd
import numpy as np

N_samples = int(1e4)
N_nan = N_var = int(0.02 * N_samples)

# Generate random data
data = np.random.rand(N_samples,2) * [30, 360]
data[np.random.choice(N_samples, N_nan), 1] = np.nan
data[np.random.choice(N_samples, N_var), 1] = 990

# Create dataset
df = pd.DataFrame(data, columns=['WindSpeed', 'WindDir'])
df.index = pd.date_range(start='2000-01-01 00:00', periods=N_samples, freq='10min')

In [16]:
df

,WindSpeed,WindDir
2000-01-01 00:00:00,16.167381,311.476589
2000-01-01 00:10:00,18.632436,115.963904
2000-01-01 00:20:00,14.651068,188.463389
2000-01-01 00:30:00,7.614744,266.786817
2000-01-01 00:40:00,27.359652,201.191199
...,...,...
2000-03-10 09:50:00,25.157903,155.537917
2000-03-10 10:00:00,6.882836,46.315722
2000-03-10 10:10:00,1.121534,232.981244
2000-03-10 10:20:00,24.099449,180.825520


In [17]:
def meandir(x):
    '''
    Parameters
    ----------
    x : pandas.Series
        pandas series to be averaged

    Returns
    -------
    float
        averaged wind direction
    '''

    # Removes the NaN from the recording
    x = x.dropna()

    # If the record is empty, return NaN
    if len(x)==0:
        return np.nan

    # If the record contains variable samples (990) return variable (990)
    elif np.any(x == 990):
        return 990

    # Otherwise sum the vectors and return the angle
    else:
        angle = np.rad2deg(
                           np.arctan2(
                                   np.sum(np.sin(np.deg2rad(x))),
                                   np.sum(np.cos(np.deg2rad(x)))
                                     )
                          )

        #Wrap angles from (-pi,pi) to (0,360)
        return (angle + 360) % 360

In [20]:
grouped = df.groupby(pd.Grouper(freq='H'))   # Data from 14.30 to 15.29 are rounded to 15.00
aggfuns1 = {'WindSpeed': np.mean, 'WindDir':meandir}
aggfuns2 = {'WindSpeed': np.mean, 'WindDir':np.mean}

res = repeat(stmt='grouped.agg(aggfuns1)', globals=globals(), number=1, repeat=10)
print(f'With custom aggregating function {min(res)*1000:.2f} ms')

res = repeat(stmt='grouped.agg(aggfuns2)', globals=globals(), number=1, repeat=10)
print(f'Without custom aggregating function {min(res)*1000:.2f} ms')

With custom aggregating function 1175.76 ms
Without custom aggregating function 1.65 ms


,WindSpeed,WindDir
2000-01-01 00:00:00,16.167381,311.476589
2000-01-01 00:10:00,18.632436,115.963904
2000-01-01 00:20:00,14.651068,188.463389
2000-01-01 00:30:00,7.614744,266.786817
2000-01-01 00:40:00,27.359652,201.191199
...,...,...
2000-03-10 09:50:00,25.157903,155.537917
2000-03-10 10:00:00,6.882836,46.315722
2000-03-10 10:10:00,1.121534,232.981244
2000-03-10 10:20:00,24.099449,180.825520


In [35]:
set([i[1].shape for i in df.dropna().groupby(pd.Grouper(freq='H'))])

{(4, 2), (5, 2), (6, 2)}

In [22]:
pd.Grouper()

,WindSpeed,WindDir
2000-01-01 00:00:00,16.167381,311.476589
2000-01-01 00:10:00,18.632436,115.963904
2000-01-01 00:20:00,14.651068,188.463389
2000-01-01 00:30:00,7.614744,266.786817
2000-01-01 00:40:00,27.359652,201.191199
...,...,...
2000-03-10 09:50:00,25.157903,155.537917
2000-03-10 10:00:00,6.882836,46.315722
2000-03-10 10:10:00,1.121534,232.981244
2000-03-10 10:20:00,24.099449,180.825520


In [21]:
grouped.agg(aggfuns1)

,WindSpeed,WindDir
2000-01-01 00:00:00,14.238001,243.646108
2000-01-01 01:00:00,15.045553,129.394194
2000-01-01 02:00:00,15.771692,990.000000
2000-01-01 03:00:00,18.017857,104.745571
2000-01-01 04:00:00,17.057802,283.872748
...,...,...
2000-03-10 06:00:00,17.263978,125.068872
2000-03-10 07:00:00,19.444256,111.211559
2000-03-10 08:00:00,19.298598,332.423324
2000-03-10 09:00:00,16.144978,213.599011
